# Inteligencia Artificial Explicable

<a target="_blank" href="https://colab.research.google.com/github/griverat/Meteo-AI/blob/main/notebooks/6.xAI.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

**Si usa Google Colab, asegúrese de tener habilitada la GPU para este notebook.**

![gpu_colab](https://github.com/griverat/Meteo-AI/blob/main/images/colab_gpu.png?raw=1)

## Descrición

En este notebook se presenta un ejemplo de cómo se puede explicar un modelo de inteligencia artificial. Para ello, se utilizará un modelo simple y se explicará su comportamiento a través de multiples técnicas de explicabilidad.


## Objetivos

- Entender cómo se puede explicar un modelo de inteligencia artificial.
- Conocer las diferentes técnicas de explicabilidad.
- Aplicar las técnicas de explicabilidad a un modelo simple.

---

In [ ]:
!pip install innvestigate

### Importar librerías

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

plt.rcParams["font.family"] = "monospace"

## Inteliencia Artificial Explicable

La inteligencia artificial explicable (XAI) es un campo de la inteligencia artificial (IA) que se preocupa por hacer que los sistemas de IA sean comprensibles para los seres humanos. La explicabilidad es importante porque los sistemas de IA pueden ser complejos y difíciles de entender, lo que puede hacer que sea difícil para los humanos confiar en ellos y tomar decisiones basadas en ellos.

En el ámbito de ciencias del clima, la explicabilidad nos puede ayudar a entender cómo los modelos de IA toman decisiones y a interpretar sus resultados. Esto es importante porque nos puede ayudar a encontrar relaciones espacio-temporales en los datos y a entender cómo los modelos de IA pueden ser utilizados para predecir el clima.

Para comenzar, necesitamos un modelo que podamos explicar. En este caso, utilizaremos primero un modelo pre-entrenado de clasificación de imágenes para obtener una idea general de como se usa xAI en la práctica.

In [ ]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

El modelo convolucional simple es suficiente para este propósito.

In [ ]:
model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(10, activation=None),
    ]
)
model.summary()

Ahora entrenamos y visualizamos las métricas de nuestro modelo.

In [ ]:
model.compile(
    optimizer="adam",
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, epochs=10, validation_split=0.2, batch_size=128)

In [ ]:
history = pd.DataFrame(history.history)
history[["loss", "val_loss"]].plot()

In [ ]:
preds = model.predict(x_test).argmax(axis=1)

In [ ]:
model.save("mnist_cnn.keras")

## Análisis de la explicabilidad

Ahora vamos a explorar el uso de los métodos Integrated Gradients y LRP para explicar el modelo de clasificación de imágenes.

El primer paso requerido por el paquete `innvestigate` es deshabilitar la ejecución de TensorFlow en modo eager. Esto quiere decir que las operaciones de TensorFlow se ejecutarán en modo gráfico, lo que permite a `innvestigate` analizar la estructura del modelo.

In [ ]:
import innvestigate

Debemos tener en cuenta que `innvestigate` funciona solo cuando la ultima capa del modelo no tiene activación, debido a ello habiamos definido previamente la capa de salida como `None`.

Ahora crearemos el analizador.

In [ ]:
with tf.compat.v1.Session() as sess:
    model = tf.keras.models.load_model("mnist_cnn.keras")
    idx = np.random.randint(0, 10000, 10)
    analyzer = innvestigate.create_analyzer("integrated_gradients", model)
    analysis = np.array(
        [analyzer.analyze(_input.reshape(1, 28, 28, 1)) for _input in x_test[idx]]
    ).squeeze()

Ahora vamos a analizar una imagen de ejemplo con el modelo y el analizador.

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(5, 20))
for i, j in enumerate(idx):
    axs[i, 0].imshow(x_test[j, :, :, 0], cmap="gray")
    axs[i, 0].axis("off")
    axs[i, 0].set_title(f"True: {y_test[j]}\nPred: {preds[j]}")
    axs[i, 1].imshow(analysis[i], cmap="seismic", vmin=-0.01, vmax=0.01)
    axs[i, 1].axis("off")
    axs[i, 1].set_title("Integrated Gradients")

fig.tight_layout()

Ahora vamos a explorar con el LRP

In [ ]:
with tf.compat.v1.Session() as sess:
    model = tf.keras.models.load_model("mnist_cnn.keras")
    analyzer_lrp = innvestigate.create_analyzer("lrp.sequential_preset_a", model)
    analysis_lrp = np.array(
        [analyzer_lrp.analyze(_input.reshape(1, 28, 28, 1)) for _input in x_test[idx]]
    ).squeeze()

In [ ]:
fig, axs = plt.subplots(10, 2, figsize=(5, 20))
for i, j in enumerate(idx):
    axs[i, 0].imshow(x_test[j, :, :, 0], cmap="gray")
    axs[i, 0].axis("off")
    axs[i, 0].set_title(f"True: {y_test[j]}\nPred: {preds[j]}")
    axs[i, 1].imshow(analysis_lrp[i], cmap="seismic", vmin=-0.0001, vmax=0.0001)
    axs[i, 1].axis("off")
    axs[i, 1].set_title("LRP")

fig.tight_layout()

Combinando todo lado a lado

In [ ]:
with tf.compat.v1.Session() as sess:
    model = tf.keras.models.load_model("mnist_cnn.keras")
    idx = np.random.randint(0, 10000, 10)
    analyzer = innvestigate.create_analyzer("integrated_gradients", model)
    analysis = np.array(
        [analyzer.analyze(_input.reshape(1, 28, 28, 1)) for _input in x_test[idx]]
    ).squeeze()

    analyzer_lrp = innvestigate.create_analyzer("lrp.sequential_preset_a", model)
    analysis_lrp = np.array(
        [analyzer_lrp.analyze(_input.reshape(1, 28, 28, 1)) for _input in x_test[idx]]
    ).squeeze()

In [ ]:
fig, axs = plt.subplots(10, 3, figsize=(7.5, 20))
for i, j in enumerate(idx):
    axs[i, 0].imshow(x_test[j, :, :, 0], cmap="gray")
    axs[i, 0].axis("off")
    axs[i, 0].set_title(f"True: {y_test[j]}\nPred: {preds[j]}")
    axs[i, 1].imshow(analysis[i], cmap="seismic", vmin=-0.01, vmax=0.01)
    axs[i, 1].axis("off")
    axs[i, 1].set_title("Integrated Gradients")
    axs[i, 2].imshow(analysis_lrp[i], cmap="seismic", vmin=-0.0001, vmax=0.0001)
    axs[i, 2].axis("off")
    axs[i, 2].set_title("LRP")

fig.tight_layout()